# ISBI CTC 2D Datasets

Download and unzip data from the following urls int the `data/ctc` folder

In [11]:
import glob
import os

import numpy as np
import pandas as pd

In [5]:
root_dir = '../data/ctc'

In [67]:
datasets = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]

In [68]:
datasets

['BF-C2DL-HSC',
 'Fluo-C2DL-MSC',
 'Fluo-N2DH-GOWT1',
 'BF-C2DL-MuSC',
 'PhC-C2DH-U373',
 'PhC-C2DL-PSC',
 'Fluo-N2DL-HeLa',
 'Fluo-N2DH-SIM+',
 'DIC-C2DH-HeLa',
 'Fluo-C2DL-Huh7']

In [69]:
rows = []
for d in datasets:
    modality, meta, cell = d.split('-')
    annotation = meta[0]
    
    for mov in glob.glob(f'{root_dir}/{d}/*_GT'):
        track = pd.read_csv(f'{mov}/TRA/man_track.txt', names=['L', 'B', 'E', 'P'], sep=' ')
        
        track_count = len(track)
        
        # Extract rows with parent cells aka division
        divisions = track[track['P']!=0]
        # Count number of unique parents
        div_count = len(np.unique(divisions['P']))
        
        rows.append({
            'modality': modality,
            'annotation': annotation,
            'celltype': cell,
            'divisions': div_count,
            'tracks': track_count
        })
        
df = pd.DataFrame(rows)

In [70]:
df.groupby(['modality', 'annotation']).sum()

/var/folders/bd/bn434f8n1bsdd8y0yby_fv7r0000gn/T/ipykernel_33953/1745371177.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby(['modality', 'annotation']).sum()


divisions  tracks
modality annotation                   
BF       C                 242     459
DIC      C                  18      70
Fluo     C                  10     128
         N                 395    1227
PhC      C                1019    2415

In [71]:
df.groupby(['modality', 'annotation'])['celltype'].nunique()

modality  annotation
BF        C             2
DIC       C             1
Fluo      C             2
          N             3
PhC       C             2
Name: celltype, dtype: int64